## FABlib API References Examples

- [fablib.show_config](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.show_config)
- [fablib.list_sites](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.list_sites)
- [fablib.list_hosts](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.list_hosts)
- [fablib.new_slice](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.new_slice)
- [slice.add_node](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.add_node)
- [slice.submit](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.submit)
- [slice.get_nodes](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.get_nodes)
- [slice.list_nodes](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.list_nodesß)
- [slice.show](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.show)
- [node.execute](https://fabric-fablib.readthedocs.io/en/latest/node.html#fabrictestbed_extensions.fablib.node.Node.execute)
- [slice.delete](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.delete) 

In [1]:
import datetime

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,49f65ad7-d8a2-4ab9-8ca0-ba777a2e0ea2
Bastion Host,bastion.fabric-testbed.net
Bastion Username,jakejongejans_0000313927
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key


In [2]:
slice_name = 'DYNAMOS-on-FABRIC'
image = "default_ubuntu_24"

# Host of the site used for the nodes (Use specific host to avoid error occurred without it: "Timeout waiting for the server to come up")
site = "AMST"

site_host = "amst-w3.fabric-testbed.net"

# Cores, RAM, storage
control_settings = [4, 12, 25]
dynamos_settings = [4, 12, 25]
worker_settings = [8, 24, 25]
worker_count = 3

In [3]:
# Create a slice
slice = fablib.new_slice(name=slice_name)

# Add Nodes with the specific variables
# Also validate the node can be created and raise an exception in case of failure
print('Adding control node...')
nodes = [
    slice.add_node(name='control', site=site, cores=control_settings[0], ram=control_settings[1], disk=control_settings[2], validate=True, raise_exception=True, host=site_host, image=image),
    slice.add_node(name='dynamos', site=site, cores=dynamos_settings[0], ram=dynamos_settings[1], disk=dynamos_settings[2], validate=True, raise_exception=True, host=site_host, image=image)
]

for i in range(1, worker_count + 1):
    print(f'Adding worker node {i}...')
    nodes.append(
        slice.add_node(name=f'worker-{i}', site=site, cores=worker_settings[0], ram=worker_settings[1], disk=worker_settings[2], validate=True, raise_exception=True, host=site_host, image=image)
    )

print('Adding network interfaces...')
interfaces = [
    node.add_component(model='NIC_Basic', name='NIC').get_interfaces()[0]
    for node in nodes
]

print('Adding network...')
network = slice.add_l3network(name='Network', interfaces=interfaces, type='IPv4')

# Calculate the lease end time for 2 weeks from now with timezone information
lease_end_time = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(weeks=2)

# Submit the slice, using an end date 2 weeks from now (the current maximum lease time) 
# to make sure that the slice can be used for a longer period of time. Progress shows an indicator of the current progression.
# Wait until the state is finished and use an interval (it may take some time before the slice and nodes are created)
print('Creating slice...')
slice.submit(wait=True, wait_timeout=3600, wait_interval=20, progress=True, wait_jupyter='text', lease_end_time=lease_end_time);


Retry: 11, Time: 299 sec


ID,13e7d294-851a-4739-bef0-5f6fbdb483b7
Name,DYNAMOS-on-FABRIC
Lease Expiration (UTC),2025-05-01 08:55:02 +0000
Lease Start (UTC),2025-04-17 08:55:02 +0000
Project ID,49f65ad7-d8a2-4ab9-8ca0-ba777a2e0ea2
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
e173a5e6-c588-4e49-9488-66526ae325b0,control,4,16,100,default_ubuntu_24,qcow2,amst-w3.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fe25:7a7d,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fe25:7a7d,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
08235639-7677-4740-8c91-b19d6bba8754,dynamos,4,16,100,default_ubuntu_24,qcow2,amst-w3.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fe69:e6c4,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fe69:e6c4,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
2e79b77a-54e4-47ed-b3f9-b267ed6c50a0,worker-1,8,32,100,default_ubuntu_24,qcow2,amst-w3.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fe3c:ff12,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fe3c:ff12,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
cd23f947-12d1-4b00-8a0b-0182d50dd401,worker-2,8,32,100,default_ubuntu_24,qcow2,amst-w3.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fecf:7bc5,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fecf:7bc5,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
2586f7d1-ef32-493a-9f8b-af67c890797d,worker-3,8,32,100,default_ubuntu_24,qcow2,amst-w3.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fe8c:840e,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fe8c:840e,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
6fe99089-b9e8-40b0-95aa-cceb46ecf794,Network,L3,FABNetv4,AMST,10.145.3.0/24,10.145.3.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
control-NIC-p1,p1,control,Network,100,config,,0A:40:F0:AF:E6:92,enp7s0,enp7s0,fe80::840:f0ff:feaf:e692,4,HundredGigE0/0/0/9
dynamos-NIC-p1,p1,dynamos,Network,100,config,,0A:B1:A5:3F:0F:02,enp7s0,enp7s0,fe80::8b1:a5ff:fe3f:f02,4,HundredGigE0/0/0/9
worker-1-NIC-p1,p1,worker-1,Network,100,config,,0A:72:73:1C:13:2A,enp7s0,enp7s0,fe80::872:73ff:fe1c:132a,4,HundredGigE0/0/0/9
worker-2-NIC-p1,p1,worker-2,Network,100,config,,0E:7D:85:09:83:D6,enp7s0,enp7s0,fe80::c7d:85ff:fe09:83d6,4,HundredGigE0/0/0/9
worker-3-NIC-p1,p1,worker-3,Network,100,config,,0E:4F:18:21:9F:35,enp7s0,enp7s0,fe80::c4f:18ff:fe21:9f35,4,HundredGigE0/0/0/9



Time to print interfaces 316 seconds


In [4]:
slice.list_nodes();

ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
e173a5e6-c588-4e49-9488-66526ae325b0,control,4,16,100,default_ubuntu_24,qcow2,amst-w3.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fe25:7a7d,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fe25:7a7d,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
08235639-7677-4740-8c91-b19d6bba8754,dynamos,4,16,100,default_ubuntu_24,qcow2,amst-w3.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fe69:e6c4,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fe69:e6c4,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
2e79b77a-54e4-47ed-b3f9-b267ed6c50a0,worker-1,8,32,100,default_ubuntu_24,qcow2,amst-w3.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fe3c:ff12,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fe3c:ff12,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
cd23f947-12d1-4b00-8a0b-0182d50dd401,worker-2,8,32,100,default_ubuntu_24,qcow2,amst-w3.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fecf:7bc5,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fecf:7bc5,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
2586f7d1-ef32-493a-9f8b-af67c890797d,worker-3,8,32,100,default_ubuntu_24,qcow2,amst-w3.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fe8c:840e,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fe8c:840e,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


In [5]:
for node in slice.get_nodes():
    stdout, stderr = node.execute('echo Hello, FABRIC from node `hostname -s`')

Hello, FABRIC from node control
Hello, FABRIC from node dynamos
Hello, FABRIC from node worker-1
Hello, FABRIC from node worker-2
Hello, FABRIC from node worker-3
